In [12]:
from minio.error import S3Error
from datetime import datetime
from io import BytesIO
import pandas as pd
import import_ipynb
import sys
import os
from dotenv import load_dotenv


In [ ]:
# Verificar ubicación y lectura del archivo Utils.ipynb
notebook_dir = os.getcwd()
utils_path = os.path.join(notebook_dir, 'Utils.ipynb')

print("📂 Directorio actual:", notebook_dir)
print("📄 Intentando acceder a:", utils_path)

# Validar que el archivo se pueda abrir
if not os.path.exists(utils_path):
    raise FileNotFoundError("❌ Utils.ipynb no se encuentra en el directorio actual.")
elif not os.access(utils_path, os.R_OK):
    raise PermissionError("❌ El archivo Utils.ipynb existe pero no tiene permisos de lectura.")

# Asegura que esté en el path para importación
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

📂 Directorio actual: d:\Carlos\Udenar-Investigación\Tesis\Extraccion Egresos CANREC\notebookPdf\Notebooks\Notebooks\Notebooks\notebooks_etl\Landing-Bronze
📄 Intentando acceder a: d:\Carlos\Udenar-Investigación\Tesis\Extraccion Egresos CANREC\notebookPdf\Notebooks\Notebooks\Notebooks\notebooks_etl\Landing-Bronze\Utils.ipynb


In [11]:
import import_ipynb
from Utils import conectar_minio, guardar_df_en_minio, generar_ruta_fecha, extraer_archivo_minio

In [ ]:
print("Utils.ipynb existe:", os.path.exists(os.path.join(notebook_dir, 'Utils.ipynb')))

Utils.ipynb existe: True
Utils.py existe: False


In [2]:
# Cargar variables del archivo .env
load_dotenv()

True

In [3]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [4]:
# Extraer un archivo Excel/CSV/Parquet
df_datos = extraer_archivo_minio(
    minio_client=minio_client,
    bucket_name="yachay-landing",
    ruta_archivo="Laboratorios/Laboratorio Citopat/informe anatomo-patologico.xlsx",
    tipo_archivo="excel"
)
print(df_datos.head())  # Mostrar las primeras filas del DataFrame extraído

   PROTOCOLO      CEDULA                   NOMBRE  EDAD SEXO  TELEFONO  \
0      12023    27451216  ESPERANZA NOGUERA PABON    54    F       NaN   
1      22023    37050039     MARIA FLORALBA ERAZO    67    F       NaN   
2      32023    30730297    AMPARO ROSERO MIDEROS    60    F       NaN   
3      42023  1108338133         JUAN DAVID MUÑOZ    10    M       0.0   
4      52023    27433207      ALBA LIGIA MARTINEZ    57    F       NaN   

        MEDICO REMITENTE DIAGNOSTICO CLINICO    MATERIAL ENVIADO  RECIBIDO  \
0       DR. ALVARO MUÑOZ    GASTRITIS ANTRAL  BIOPSIAS GASTRICAS  10/01/23   
1  DR. EDGARDO BENAVIDES            PROLAPSO               UTERO  10/01/23   
2       DR. ALVARO MUÑOZ                   -  BIOPSIAS GASTRICAS  10/01/23   
3     DR. EDWARD BOLIVAR   APENDICITIS AGUDA      APENDICE CECAL  10/01/23   
4       DR. ALVARO MUÑOZ                   -  BIOPSIAS GASTRICAS  10/01/23   

  ENTREGADO                           DESCRIPCION MACROSCOPICA  \
0  17/01/23  1. Rotu

In [5]:
ruta_fecha = generar_ruta_fecha()

In [6]:
# Guardar el DataFrame en MinIO
ruta_minio = guardar_df_en_minio(
    minio_client=minio_client,  # Cliente de la conexión anterior
    df=df_datos,
    bucket_name="yachay-bronze",
    ruta_destino=f"laboratorios/{ruta_fecha}/laboratorio Citopat/citopat",  # La extensión se agregará automáticamente
    formato='csv',  # También puede ser 'csv', 'json' o 'excel'
    crear_bucket=True
)

DataFrame guardado exitosamente en: yachay-bronze/laboratorios/2025/06/15/laboratorio Citopat/citopat.csv
